In [3]:
%load_ext lab_black

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [4]:
def run_docker_container():
    print(f"Current working directory: {os.getcwd()}")  # Add this line
    cmd = f'docker run -v "{os.getcwd()}/data:/app/data" --name nfl-injury-scraper-container nfl-injury-scraper'
    print(f"Running Docker command: {cmd}")  # Add this line
    # ...

In [5]:
run_docker_container()

Current working directory: /mnt/batch/tasks/shared/LS_root/mounts/clusters/matthewhanauer991/code/Users/matthewhanauer99/draft_kings_analyses/src
Running Docker command: docker run -v "/mnt/batch/tasks/shared/LS_root/mounts/clusters/matthewhanauer991/code/Users/matthewhanauer99/draft_kings_analyses/src/data:/app/data" --name nfl-injury-scraper-container nfl-injury-scraper


Step: Load data

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import numpy as np
from pyprojroot import here
from skimpy import clean_columns
import os

path_data = here("./data")
os.chdir(path_data)

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

url = "https://www.espn.com/nfl/injuries"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables containing injury data
injury_tables = soup.find_all("table", class_="Table")

# Define the columns for the DataFrame
columns = ["Name", "POS", "Date", "Status"]

# Prepare an empty list to store the data
injury_data = []

# Get the current year
current_year = datetime.datetime.now().year

# Iterate over each table
for injury_table in injury_tables:
    # Get all rows in the table
    rows = injury_table.find_all("tr")

    # Iterate over the rows, excluding the header row
    for row in rows[1:]:
        cells = row.find_all("td")

        # Extract data from the cells
        name = cells[0].text.strip()
        pos = cells[1].text.strip()
        date = cells[2].text.strip() + f" {current_year}"
        status = cells[3].text.strip()

        # Append the data as a tuple to the injury_data list
        injury_data.append((name, pos, date, status))

# Create the DataFrame
injury_df = pd.DataFrame(injury_data, columns=columns)

print(injury_df)

                     Name POS         Date        Status
0        Garrett Williams  CB  Apr 28 2023  Questionable
1            Kyler Murray  QB   Apr 7 2023  Questionable
2          D.J. Humphries  OT  Mar 10 2023  Questionable
3              Colt McCoy  QB  Feb 24 2023  Questionable
4               Zach Ertz  TE  Feb 18 2023  Questionable
..                    ...  ..          ...           ...
276  James Smith-Williams  DE  Jan 10 2023  Questionable
277    Brian Robinson Jr.  RB  Jan 10 2023  Questionable
278        Andrew Norwell   G  Jan 10 2023  Questionable
279       Cornelius Lucas  OT  Jan 10 2023  Questionable
280         John Ridgeway  DT   Jan 9 2023  Questionable

[281 rows x 4 columns]


In [35]:
data_nfl_injury = clean_columns(injury_df)

4 column names have been cleaned

In [36]:
data_nfl_injury["date"] = pd.to_datetime(data_nfl_injury["date"])

data_nfl_injury

name pos       date        status
0        Garrett Williams  CB 2023-04-28  Questionable
1            Kyler Murray  QB 2023-04-07  Questionable
2          D.J. Humphries  OT 2023-03-10  Questionable
3              Colt McCoy  QB 2023-02-24  Questionable
4               Zach Ertz  TE 2023-02-18  Questionable
..                    ...  ..        ...           ...
276  James Smith-Williams  DE 2023-01-10  Questionable
277    Brian Robinson Jr.  RB 2023-01-10  Questionable
278        Andrew Norwell   G 2023-01-10  Questionable
279       Cornelius Lucas  OT 2023-01-10  Questionable
280         John Ridgeway  DT 2023-01-09  Questionable

[281 rows x 4 columns]

In [37]:
data_nfl_injury.to_parquet("data_nfl_injury.parquet")